# Main notebook for data cleaning and insertion into database

In [ ]:
# standard library imports
import os
import sqlite3
import sys

sys.path.append(os.path.join(os.path.dirname("__file__"), "..", ".."))

# third party imports
import pandas as pd

# local imports
from src.databases import DatabaseCreator

## UFC Stats

Cleaning was all done inside the corresponding Scrapy spider.

## Wikipedia

Cleaning was all done inside the corresponding Scrapy spider.